In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import os
from lib import model_device_io as io
from lib import data_loader as dl
from lib import models as md

In [21]:
trainset = dl.HotelReservationData(root='data/train')
testset = dl.HotelReservationData(root='data/test')
train_loader = DataLoader(trainset, batch_size=4096, shuffle=True, num_workers=0)
test_loader = DataLoader(testset, batch_size=4096, shuffle=False, num_workers=0)
device = io.getCudaDevice(cudaNum = 1, torchSeed = 123)
model = md.ClassifierLinear(input_size = 258).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
io.loadModel('models/rev_best.pth', model, optimizer)

Device used: cuda:1
model loaded from models/rev_best.pth


In [22]:
def train(train_epoch = 100, model_fn = "rev_best.pth", log_per_epoch = 1):
    best_acc = test()
    for epoch in range(train_epoch):
        model.train()
        for batch_idx, (data, _, rev) in enumerate(train_loader):
            data, rev = data.to(device), rev.to(device)
            output = model(data)
            loss = F.cross_entropy(output, rev)
            loss.backward()
            optimizer.step()
            if (batch_idx+1) % max(1, int(len(train_loader)/log_per_epoch)) == 0:
                print('Training Epoch: {:.2f}% ({}/{})\tBatch: {:.2f}% ({}/{})\tLoss: {:.6f}'.format(
                      (epoch+1)*100./train_epoch, epoch+1, train_epoch, (batch_idx+1)*100./len(train_loader),
                      batch_idx+1, len(train_loader), loss.item()))
        acc = test()
        if(best_acc < acc):
            best_acc = acc
            fn = os.path.join('models', model_fn)
            io.saveModel(fn, model, optimizer)
            
            

def test():
    model.eval()
    loss, correct = 0, 0
    with torch.no_grad(): # This will free the GPU memory used for back-prop
        for data, _, rev in test_loader:
            data, rev = data.to(device), rev.to(device)
            output = model(data)
            loss += F.cross_entropy(output, rev).item()*len(data) # sum up batch loss
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(rev.view_as(pred)).sum().item()
            
    loss /= len(test_loader.dataset)
    acc = 1.*correct/len(test_loader.dataset)
    print('***Validation Results***\tLoss: {:.4f},\tAccuracy: {:.2f}% ({}/{})\n'.format(
          loss, acc*100., correct, len(test_loader.dataset)))
    return acc

In [ ]:
train(train_epoch = 5000)

***Validation Results***	Loss: 1518.3077,	Accuracy: 69.32% (6932/10000)

Training Epoch: 0.02% (1/5000)	Batch: 100.00% (20/20)	Loss: 1425.119873
***Validation Results***	Loss: 1801.9521,	Accuracy: 66.96% (6696/10000)

Training Epoch: 0.04% (2/5000)	Batch: 100.00% (20/20)	Loss: 1712.147827
***Validation Results***	Loss: 1690.7118,	Accuracy: 68.89% (6889/10000)

Training Epoch: 0.06% (3/5000)	Batch: 100.00% (20/20)	Loss: 1442.703857
***Validation Results***	Loss: 1463.3652,	Accuracy: 68.62% (6862/10000)

Training Epoch: 0.08% (4/5000)	Batch: 100.00% (20/20)	Loss: 1713.805176
***Validation Results***	Loss: 1779.1821,	Accuracy: 44.00% (4400/10000)

Training Epoch: 0.10% (5/5000)	Batch: 100.00% (20/20)	Loss: 1655.023682
***Validation Results***	Loss: 2261.0960,	Accuracy: 38.75% (3875/10000)

Training Epoch: 0.12% (6/5000)	Batch: 100.00% (20/20)	Loss: 1963.965576
***Validation Results***	Loss: 2230.6133,	Accuracy: 38.96% (3896/10000)

Training Epoch: 0.14% (7/5000)	Batch: 100.00% (20/20)	Los